In [1]:
import resfuncRead as rfr
import numpy as np
from Yield_calc import * 
import pandas as pd 

from tabulate import tabulate

bpar_er = rfr.getBandFunc('data/erband_R133') #reads in band data/fit for er data
bpar_nr = rfr.getBandFunc('data/nrband_R133')

ynr_mu = rfr.makeBFunc(bpar_nr[1]['mu'])
ynr_muv = np.vectorize(ynr_mu)
ynr_sig = rfr.makeBFunc(bpar_nr[1]['sig'],True)
ynr_sigv = np.vectorize(ynr_sig)

yer_mu = rfr.makeBFunc(bpar_er[1]['mu']) # sets average fit from 1st (2nd) col in data table. 
yer_muv = np.vectorize(yer_mu) #puts mean data for er in 1D array 
yer_sig = rfr.makeBFunc(bpar_er[1]['sig'],True) #sets uncertainty 
yer_sigv = np.vectorize(yer_sig)





In [2]:
s = 1
N = 10000
ER, Yield = Yield_NR(N)
Erer, Yield_er = Yield_Er(N)

#for nuclear recoils 
upper = ynr_mu(ER)+s*ynr_sigv(ER)
lower = ynr_mu(ER)-s*ynr_sigv(ER)

Data = np.vstack((ER,Yield,upper,lower)).T




In [3]:
Data1 = Data[np.argsort(Data[:, 0])]
'''
print(Data)
print('--------------------------------------------------------')
print((Data1))
'''

"\nprint(Data)\nprint('--------------------------------------------------------')\nprint((Data1))\n"

In [4]:
df = pd.DataFrame(Data1)

bins  = [10,13.4,18.1,24.5,33.1,44.8,60.6,80.2,110,150,200]

df1 = pd.cut(df[0],bins, labels = False)


In [6]:
Percent = []

for q in np.arange(0,9):
    
    count = []
    num = []
    count1 = []
    num1 = []
    
    
    x = np.array(df[1].loc[df1 == q])
    y = np.array(df[2].loc[df1 == q])
    z = np.array(df[3].loc[df1 == q])
    
    
    for i,j in zip(x,y):
        if i >= j:
            n = 1
        else: 
            n = 0 

        count.append(n)
    #print(count)

    for i in count: 
        if i == 1:
            num.append(i) 


    for i,j in zip(x,z):
        if i <= j:
            n = 1
        else: 
            n = 0 

        count1.append(n)
    #print(count)

    for i in count: 
        if i == 1:
            num1.append(i)


    N= len(x)
    percent = 100*(N - np.abs(len(num)+len(num1)))/N

    Percent.append(percent)


In [7]:
bin_spacing = '10.4-13.4','13.4-18.1','18.1-24.5','24.5-33.1','33.1-44.8','44.8-60.6','60.6-80.2','80.2-110','110-150','150-200'

In [8]:
print("Bin Spacing (keV)", '\t', "Percent in",s,"sigma")     #table column headings
print("--------------", '\t', '\t' "-------------------")

for x,y in zip(bin_spacing,Percent):
    print(x, '\t','\t', y)


Bin Spacing (keV) 	 Percent in 1 sigma
-------------- 	 	-------------------
10.4-13.4 	 	 79.37584803256445
13.4-18.1 	 	 80.80903104421449
18.1-24.5 	 	 80.76595744680851
24.5-33.1 	 	 85.02202643171806
33.1-44.8 	 	 84.05524168236033
44.8-60.6 	 	 84.78260869565217
60.6-80.2 	 	 82.72552783109406
80.2-110 	 	 77.51479289940828
110-150 	 	 70.75098814229248
